In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import yfinance as yf
import time

In [4]:
WIKI_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

def scrape_wikipedia_sp500():
    """Scrape S&P 500 companies and their sectors from Wikipedia."""
    response = requests.get(WIKI_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    table = soup.find('table', {'id': 'constituents'})
    rows = table.find_all('tr')[1:]  # Skip header row
    
    companies_data = []
    for row in rows:
        cols = row.find_all('td')
        ticker = cols[0].text.strip()
        company_name = cols[1].text.strip()
        sector = cols[3].text.strip()
        companies_data.append((ticker, company_name, sector))
    
    return companies_data

In [6]:




def get_sp500_sectors(companies_data):
    """Return a list of unique sectors."""
    return list(set(company[2] for company in companies_data))

def get_sector_companies(sector, companies_data):
    """Return the list of companies belonging to a sector."""
    return [company[0] for company in companies_data if company[2] == sector]

def download_market_cap_data(ticker, start_date, end_date):
    """Download the stock's market capitalization at the quarter end."""
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(start=start_date, end=end_date)
        if df.empty:
            print(f"No data found for {ticker}")
            return None

        # Calculate Market Cap = Close Price * Shares Outstanding
        shares_outstanding = stock.info.get('sharesOutstanding', None)
        if shares_outstanding is None:
            print(f"No shares outstanding data for {ticker}")
            return None
        
        df['MarketCap'] = df['Close'] * shares_outstanding
        df = df[['MarketCap']]  # Only keep MarketCap column
        
        # Resample to get the last value at the end of each quarter
        df = df.resample('QE').last()
        df['Ticker'] = ticker  # Add ticker as a column
        return df
    except Exception as e:
        print(f"Error downloading data for {ticker}: {str(e)}")
        return None

def process_sector(sector, start_date, end_date, companies_data):
    """Process each sector and calculate the market cap at quarter end."""
    companies = get_sector_companies(sector, companies_data)
    if len(companies) < 3:
        print(f"Skipping {sector} sector: Only {len(companies)} companies found.")
        return None

    print(f"Processing {sector} sector ({len(companies)} companies)...")
    print(companies)
    results = []

    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_ticker = {executor.submit(download_market_cap_data, ticker, start_date, end_date): ticker for ticker in companies}
        for future in as_completed(future_to_ticker):
            ticker, result = future_to_ticker[future], future.result()
            if result is not None and not result.empty:
                results.append(result)
            time.sleep(2)  # To avoid overwhelming the Yahoo Finance API

    if not results:
        print(f"No valid results for {sector} sector.")
        return None

    try:
        # Combine the results into a single DataFrame
        combined_results = pd.concat(results)
        combined_results.reset_index(inplace=True)  # Reset index to get Date as a column
        combined_results.rename(columns={'index': 'Date'}, inplace=True)  # Rename the index column to Date
        return combined_results
    except ValueError as e:
        print(f"Error combining results for {sector} sector: {str(e)}")
        return None

def main():
    # Set the date range from Q1 2015 to Q2 2024 (or current date if earlier)
    start_date = pd.Timestamp('2015-01-01')
    end_date = pd.Timestamp('2024-06-30')
    end_date = min(end_date, pd.Timestamp.now())

    # Scrape Wikipedia for S&P 500 companies and sectors
    companies_data = scrape_wikipedia_sp500()

    # Get all unique sectors
    sectors = get_sp500_sectors(companies_data)

    # Create a directory for output files
    output_dir = "sector_mkt_cap_results"
    os.makedirs(output_dir, exist_ok=True)

    # Process each sector
    for sector in sectors:
        sector_results = process_sector(sector, start_date, end_date, companies_data)
        if sector_results is not None:
            # Save results to CSV
            csv_filename = os.path.join(output_dir, f"{sector}_mkt_cap_quarter_end.csv")
            sector_results.to_csv(csv_filename, index=False)  # Save without index
            print(f"Results saved to {csv_filename}")

            # Display summary
            print(f"Summary for {sector} sector:")
            print(sector_results.head())
            print("\n" + "="*50 + "\n")
        else:
            print(f"No results to save for {sector} sector.")
            print("\n" + "="*50 + "\n")

    print("All sectors processed.")

if __name__ == "__main__":
    main()


Processing Pharmaceuticals sector (8 companies)...
['BMY', 'CTLT', 'JNJ', 'LLY', 'MRK', 'PFE', 'VTRS', 'ZTS']


FileNotFoundError: [Errno 2] No such file or directory: 'market_data.csv'

In [8]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import yfinance as yf
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

def scrape_wikipedia_sp500():
    """Scrape S&P 500 companies and their sectors from Wikipedia."""
    response = requests.get(WIKI_URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    table = soup.find('table', {'id': 'constituents'})
    rows = table.find_all('tr')[1:]  # Skip header row
    
    companies_data = []
    for row in rows:
        cols = row.find_all('td')
        ticker = cols[0].text.strip()
        company_name = cols[1].text.strip()
        sector = cols[3].text.strip()
        companies_data.append((ticker, company_name, sector))
    
    return companies_data

def get_sector_companies(sector, companies_data):
    """Return the list of companies belonging to a sector."""
    return [company[0] for company in companies_data if company[2] == sector]

def download_market_cap_data(ticker, start_date, end_date):
    """Download the stock's market capitalization at the quarter end."""
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(start=start_date, end=end_date)
        if df.empty:
            return None

        # Calculate Market Cap = Close Price * Shares Outstanding
        shares_outstanding = stock.info.get('sharesOutstanding', None)
        if shares_outstanding is None:
            return None
        
        df['MarketCap'] = df['Close'] * shares_outstanding
        df = df[['MarketCap']]  # Only keep MarketCap column
        
        # Resample to get the last value at the end of each quarter
        df = df.resample('QE').last()
        df['Ticker'] = ticker  # Add ticker as a column
        return df
    except Exception as e:
        return None

def calculate_returns(data):
    """Calculate quarterly returns based on market cap."""
    data['Return'] = data['MarketCap'].pct_change()
    return data.dropna(subset=['Return'])

def calculate_beta(sector_returns, market_returns):
    """Calculate beta using aligned sector and market returns."""
    # Align the series to have matching dates
    aligned_returns = sector_returns.align(market_returns, join='inner')
    aligned_sector_returns = aligned_returns[0]
    aligned_market_returns = aligned_returns[1]
    
    # Calculate covariance and beta
    covariance = np.cov(aligned_sector_returns, aligned_market_returns)[0, 1]
    market_variance = np.var(aligned_market_returns)
    beta = covariance / market_variance
    return beta


def process_sector_with_beta(sector, start_date, end_date, companies_data, market_returns):
    """Calculate quarterly returns and beta for a sector."""
    companies = get_sector_companies(sector, companies_data)
    results = []

    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_ticker = {executor.submit(download_market_cap_data, ticker, start_date, end_date): ticker for ticker in companies}
        for future in as_completed(future_to_ticker):
            result = future.result()
            if result is not None:
                results.append(result)

    if not results:
        return None

    # Combine the results into a single DataFrame
    combined_results = pd.concat(results)
    combined_results.reset_index(inplace=True)
    
    # Calculate sector returns
    sector_returns = combined_results.groupby('Date')['MarketCap'].sum()
    sector_returns = calculate_returns(sector_returns.to_frame())

    # Calculate beta
    beta_value = calculate_beta(sector_returns['Return'], market_returns)
    return beta_value

def main():
    start_date = pd.Timestamp('2015-01-01')
    end_date = pd.Timestamp('2024-06-30')
    
    # Scrape Wikipedia for S&P 500 companies and sectors
    companies_data = scrape_wikipedia_sp500()
    
    # Get unique sectors
    sectors = list(set(company[2] for company in companies_data))
    
    # Load market index data for beta calculation (assuming S&P 500)
    market_data = yf.Ticker('^GSPC').history(start=start_date, end=end_date)
    market_data = market_data.resample('QE').last()  # Quarterly end
    market_data['Return'] = market_data['Close'].pct_change().dropna()
    market_returns = market_data['Return']
    
    # Calculate and store sector betas
    sector_betas = {}
    for sector in sectors:
        beta_value = process_sector_with_beta(sector, start_date, end_date, companies_data, market_returns)
        if beta_value is not None:
            sector_betas[sector] = beta_value
    
    # Save to CSV
    beta_df = pd.DataFrame(sector_betas.items(), columns=['Sector', 'Beta'])
    beta_df.to_csv('sector_beta_values.csv', index=False)
    print("Sector beta values saved to sector_beta_values.csv")

if __name__ == "__main__":
    main()


AMTM: Data doesn't exist for startDate = 1420088400, endDate = 1719720000
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/var/folders/83/3t5wk1pn2fq83ss49zg61zf40000gn/T/ipykernel_65354/3393806807.py:69: RuntimeWarning: Degrees of freedom <= 0 for slice
  covariance = np.cov(aligned_sector_returns, aligned_market_returns)[0, 1]
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
$BF.B: possibly delisted; No price data found  (1d 2015-01-01 00:00:00 -> 2024-06-30 00:00:00)


$BF.B: possibly delisted; No price data found  (1d 2015-01-01 00:00:00 -> 2024-06-30 00:00:00)


SW: Data doesn't exist for startDate = 1420088400, endDate = 1719720000
$BRK.B: possibly delisted; No timezone found


Sector beta values saved to sector_beta_values.csv


/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/opt/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/var/folders/83/3t5wk1pn2fq83ss49zg61zf40000gn/T/ipykernel_65354/3393806807.py:69: RuntimeWarning: Degrees of freedom <= 0 for slice
  covariance = np.cov(aligned_sector_returns, aligned_market_returns)[0, 1]
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
